In [1]:
import os
import git
from pathlib import Path
from typing import List
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
from IPython.display import clear_output
import scipy
from scipy import stats
from scipy.spatial import ConvexHull
import pylustrator
from scipy.spatial import Delaunay
from scipy.spatial import distance
from sklearn.decomposition import PCA
from tqdm import tqdm
import warnings

warnings.simplefilter("ignore", RuntimeWarning)
ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
save_path_with_hull = Path(os.path.join(ROOT_DIR, 'publication', 'paper', 'CSVs', 'final_results_with_hull.pickle'))
save_path_without_hull = Path(os.path.join(ROOT_DIR, 'publication', 'paper', 'CSVs', 'final_results.csv'))
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")

In [2]:
def variance_prior(r, eta, scale=1):
    beta = (eta+1.5)/r
    var_prior = scale * scipy.special.gamma(beta + 1/r)/scipy.special.gamma(beta)
    return var_prior

def kurtosis_prior(r, eta, scale=1, fisher=True):
    beta = (eta+1.5)/r
    kurtosis = scale*3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2 # DOUBLE CHECK
    if fisher:
        return kurtosis - 3
    else:
        return kurtosis 

def find_master_dfs(root_dir: str) -> List[str]:
    root_path = Path(root_dir)
    if not root_path.exists():
        raise FileNotFoundError(f"Directory not found: {root_dir}")

    master_df_paths = []
    for current_dir, _, files in os.walk(root_path):
        if 'master_df.csv' in files:
            master_df_path = Path(os.path.join(current_dir, 'master_df.csv'))
            master_df_paths.append(str(master_df_path.absolute()))
    return master_df_paths

In [3]:
def add_hull(master_df, rEtaKsstats_dict, GROUP='group', debug=False):

    master_df_copy = master_df.copy()
    master_df_copy = master_df.set_index(GROUP)
    groups = master_df_copy.index
    master_df_copy["hull"] = ""

    for group in groups:
        if master_df_copy.loc[group, "total_samples"] < 10:
            master_df_copy.loc[group, "hull"] = np.nan
           
        else:
            drop_keys =list(rEtaKsstats_dict[group].keys())[-3:]
            if debug:
                print(drop_keys)
            pre_optimization = pd.DataFrame(rEtaKsstats_dict[group]).drop(drop_keys, axis = 1 )
            optimization = pd.DataFrame(rEtaKsstats_dict[group])[drop_keys]
            optimization = optimization.rename(columns = {"r_optimize": "r", "eta_optimize": "eta", drop_keys[-1]: "ksstat"})
            optimization = optimization.dropna()
            full_df = pre_optimization.merge(optimization, on=["r", "eta"], how="outer")
            full_df = full_df.set_index(["r", "eta"])
            full_df["ksstat"] = full_df.min(axis=1)
            full_df = full_df.reset_index()
            full_df = full_df[["r", "eta", "ksstat"]]
            full_df["1/beta"] = full_df["r"]/(full_df["eta"] + 1.5)
            full_df["log_beta"] = np.log10((full_df["eta"] + 1.5) / full_df["r"])
            MULT = 1.2
            cutoff = max(min(full_df["ksstat"]) * MULT, 0.01)
            filtered_df = full_df[full_df["ksstat"] < cutoff]
            points = np.column_stack((filtered_df["r"], filtered_df["1/beta"])) + stats.norm.rvs(size=(len(filtered_df), 2)) * 0.001  # Adding small noise for convex hull computation
            #points = np.column_stack((np.log10(filtered_df["r"]), filtered_df["log_beta"])) + stats.norm.rvs(size=(len(filtered_df), 2)) * 0.001  # Adding small noise for convex hull computation
            if len(points) < 3:
                hull=np.nan
                master_df_copy.loc[group, "intersect_roi"] = -1
            else:
                hull = ConvexHull(points)

                #x_vals = np.logspace(0, 20, 1000)
                #eta_vals = 1.5 + np.zeros_like(x_vals)
                #roi = (eta_vals) / x_vals
                
                #line_points = np.column_stack((x_vals, roi))
                #line_points = np.column_stack((np.log10(x_vals), np.log10(roi)))

                #intersect_roi = int(np.any(in_hull(line_points, hull)))
                if np.any(filtered_df["eta"] > 0) and np.any(filtered_df["eta"] < 0):
                    intersect_roi = 1
                else:
                    intersect_roi = 0
                master_df_copy.loc[group, "intersect_roi"] = intersect_roi
            master_df_copy.loc[group, "hull"] = hull

    return master_df_copy.reset_index()

In [4]:
def process_rEtaKsstats_dict(master_df_original, rEtaKsstats_dict, GROUP='group', debug=False):

    master_df = master_df_original.copy()
    master_df = master_df_original.set_index(GROUP)
    groups = master_df.index
    master_df["hull"] = ""

    for group in groups:
        if master_df.loc[group, "total_samples"] < 10:
            master_df.loc[group, "hull"] = np.nan
           
        else:
            drop_keys =list(rEtaKsstats_dict[group].keys())[-3:]
            if debug:
                print(drop_keys)
            pre_optimization = pd.DataFrame(rEtaKsstats_dict[group]).drop(drop_keys, axis = 1 )
            optimization = pd.DataFrame(rEtaKsstats_dict[group])[drop_keys]
            optimization = optimization.rename(columns = {"r_optimize": "r", "eta_optimize": "eta", drop_keys[-1]: "ksstat"})
            optimization = optimization.dropna()

            full_df = pre_optimization.merge(optimization, on=["r", "eta"], how="outer")
            full_df = full_df.set_index(["r", "eta"])
            full_df["ksstat"] = full_df.min(axis=1)
            full_df = full_df.reset_index()
            full_df = full_df[["r", "eta", "ksstat"]]
            full_df["1/beta"] = full_df["r"]/(full_df["eta"] + 1.5)
            
            MULT = 1.2
            cutoff = max(min(full_df["ksstat"]) * MULT, 0.01)
            filtered_df = full_df[full_df["ksstat"] < cutoff]
            points = np.column_stack((filtered_df["r"], filtered_df["1/beta"])) + stats.norm.rvs(size=(len(filtered_df), 2)) * 0.001  # Adding small noise for convex hull computation
            
            if len(points) < 3:
                hull=np.nan
                hull_area = 0
                intersect_roi = -1
            else:
                hull = ConvexHull(points)
                hull_area = hull.volume

                if np.any(filtered_df["eta"] > 0) and np.any(filtered_df["eta"] < 0):
                    intersect_roi = 1
                else:
                    intersect_roi = 0

            master_df.loc[group, "hull"] = hull    
            master_df.loc[group, "hull_area"] = hull_area
            master_df.loc[group, "intersect_roi"] = intersect_roi

            kurt_lower, kurt_upper = master_df.loc[group, "kurt_lower"], master_df.loc[group, "kurt_upper"]
            best_scale = master_df.loc[group, "best_scale"]

            kurt_df = full_df.copy()
            kurt_df['kurt'] = kurt_df.apply(lambda row : kurtosis_prior(r=row['r'], eta=row['eta'], scale=best_scale), axis=1)
            kurt_df['pass_kurt'] = (kurt_df['kurt'] > kurt_lower) & (kurt_df['kurt'] < kurt_upper)

            pass_kurt_anywhere = np.sum(kurt_df['pass_kurt'] > 0)
            if pass_kurt_anywhere >= 3:
                temp = kurt_df[kurt_df['pass_kurt'] == 1]
                points_kurt = np.column_stack((temp["r"], temp["1/beta"])) + stats.norm.rvs(size=(len(temp), 2)) * 0.001
                hull_kurt = ConvexHull(points_kurt)
                hull_kurt_area = hull_kurt.volume
            else:
                hull_kurt = np.nan
                hull_kurt_area = 0

            master_df.loc[group, "hull_kurt"] = hull_kurt
            master_df.loc[group, "hull_kurt_area"] = hull_kurt_area
            master_df.loc[group, "num_pass_kurt_anywhere"] = pass_kurt_anywhere
            master_df.loc[group, "pass_kurt_anywhere"] = int(pass_kurt_anywhere > 0)
            master_df.loc[group, "num_pass_kurt_intersect_hull"] = np.sum((kurt_df['pass_kurt'] == 1) & (kurt_df['ksstat'] < cutoff))
            master_df.loc[group, "pass_kurt_intersect_hull"] = int(master_df.loc[group, "num_pass_kurt_intersect_hull"] > 0)

    return master_df.reset_index()

In [5]:
all_paths = find_master_dfs(os.path.join(ROOT_DIR, "results", "case-studies"))
all_paths[:10]

['/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/scaleTesting/wavelet/full/gray/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/scaleTesting/wavelet/full/laplace/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/experiments/scaleTesting/_/samplePrior/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/syntheticMRI3D/full/wavelet/ada/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/syntheticMRI3D/full/wavelet/aad/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/syntheticMRI3D/full/wavelet/ddd/CSVs/master_df.csv',
 '/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/results/case-studies/syntheticMRI3D/full/wavelet/daa/CSVs/master_

In [6]:
relevant_cols = [
        'group', 'dataset', 'subset', 'transform', 'orientation', 'channel', 'dataset_type', 
        'obs_var', 'var_lower', 'var_upper', 
        'total_samples', 'initial_r', 'initial_eta',  'best_r', 'best_eta', 'best_scale',
        'kstest_stat_initial', 'kstest_stat_cutoff_0.05', 'kstest_stat_best',
        'obs_kurt', 'kurt_lower', 'kurt_upper', 
        'intersect_roi', 'hull', 'hull_area',
        'num_pass_kurt_anywhere', 'pass_kurt_anywhere', 'num_pass_kurt_intersect_hull', 'pass_kurt_intersect_hull', 'hull_kurt', 'hull_kurt_area',
        'param_gaussian', 'kstest_stat_gaussian', 'kstest_pval_gaussian', 
        'param_laplace', 'kstest_stat_laplace', 'kstest_pval_laplace', 
        'param_t', 'kstest_stat_t', 'kstest_pval_t', 'kstest_pval_gengamma', 
        'github_plot']

all_paths = find_master_dfs(os.path.join(ROOT_DIR, "results", "case-studies"))
all_master_dfs = []
github_plots_path = "https://github.com/yashdave003/hierarchical-bayesian-model-validation/blob/main/results/case-studies/"

for i in tqdm(range(len(all_paths))):
    path = all_paths[i]
    if 'scaleTesting' in path:
        continue
    master_df = pd.read_csv(path)
    master_df = master_df.rename(columns={master_df.columns[0]: 'group'})
    parts = Path(path).parts[-7:]
    if parts[0] == 'case-studies':
        parts = parts[1:]
    elif parts[0] == 'results':
        parts = parts[2:]
    if "MRI" in path:
        dataset, slice, transform, orientation, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = slice
        master_df['channel'] = np.nan
        master_df['orientation'] = orientation
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, slice, transform, orientation, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    elif len(parts) > 6:
        dataset, subset, transform, orientation, channel, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = subset
        master_df['channel'] = channel
        master_df['orientation'] = orientation
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, subset, transform, orientation, channel, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    elif "learned" in path:
        dataset, subset, transform, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = subset
        master_df = master_df.rename(columns={'filter_group' : 'orientation'})
        master_df['channel'] = np.nan
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, subset, transform, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]

    else:
        dataset, size, transform, channel, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = size
        master_df['channel'] = channel
        master_df['orientation'] = np.nan
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, size, transform, channel, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    
    if dataset in ['pastis', 'agriVision', 'spaceNet']:
        master_df['dataset_type'] = 'remote sensing'
    elif dataset in ['syntheticMRI2D', 'syntheticMRI3D']:
        master_df['dataset_type'] = 'medical'
    elif dataset in ['coco', 'segmentAnything', 'standardTesting']:
        master_df['dataset_type'] = 'natural'
    elif dataset in ['standardTesting']:
        master_df['dataset_type'] = 'classical'

    GROUP = 'layer' if transform.split("-")[0] == 'wavelet' else ('band' if transform.split("-")[0] == 'fourier' else 'filter_idx')
    rEtaKsstatsDict = pd.read_pickle(path[:-18] + "cache" + os.sep + "rEtaKsstats_dict.pickle")
    
    master_df = process_rEtaKsstats_dict(master_df, rEtaKsstatsDict)
    all_master_dfs.append(master_df[relevant_cols])
    
main_df = pd.concat(all_master_dfs)

main_df['best_beta'] = (main_df['best_eta'] + 1.5)/main_df['best_r'] 
main_df['best_1/beta'] = 1/main_df['best_beta']
main_df['beat_all_priors'] = (main_df['kstest_stat_best'] < np.minimum.reduce([main_df['kstest_stat_gaussian'], main_df['kstest_stat_laplace'], main_df['kstest_stat_t']])).astype(int)
main_df["best_prior"] = np.array(["GenGamma", "Gaussian", "Laplace", "Student-T", np.nan])[
                                np.nanargmin(np.array([main_df['kstest_stat_best'], 
                                                        main_df['kstest_stat_gaussian'], 
                                                        main_df['kstest_stat_laplace'], 
                                                        main_df['kstest_stat_t'], 
                                                        0.99*np.ones_like(main_df['kstest_stat_t'])]
                                                        ).T, axis=1)]

print("Main DF (before frequency):", main_df.shape)
frequency_map = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")).set_index(['dataset', 'transform', 'group'])
main_df = main_df.set_index(['dataset', 'subset', 'transform', 'group']).merge(frequency_map, left_index = True, right_index=True, how='left').reset_index()

print("Main DF (after frequency):", main_df.shape)
old_fail_cat_df_path = os.path.join(ROOT_DIR, "publication", "paper", "CSVs", 'result_categorization_sheet - combined_categories.csv')
old_fail_cat_df = pd.read_csv(old_fail_cat_df_path)
main_df = main_df.merge(old_fail_cat_df[['github_plot', 'failure_category', 'failure_type', 'which_ones']], on='github_plot', how='left')
print("Main DF (after result categorization):", main_df.shape)

  0%|          | 0/101 [00:00<?, ?it/s]/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_87989/580829178.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<scipy.spatial._qhull.ConvexHull object at 0x7fb131dc6b90>' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  master_df.loc[group, "hull_kurt"] = hull_kurt
  4%|▍         | 4/101 [00:00<00:22,  4.25it/s]/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_87989/580829178.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<scipy.spatial._qhull.ConvexHull object at 0x7fb131cf6800>' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  master_df.loc[group, "hull_kurt"] = hull_kurt
  9%|▉         | 9/101 [00:05<01:16,  1.20it/s]/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ip

Main DF (before frequency): (1155, 46)
Main DF (after frequency): (1155, 47)
Main DF (after result categorization): (1155, 50)


In [7]:
main_df['hull_area'].sort_values(ascending=False)
main_df['hull_kurt_area'].sort_values(ascending=False)

490     2090.126902
126     2070.209599
823     2041.672760
150     2024.673154
137     1991.143102
           ...     
105             NaN
113             NaN
1128            NaN
1137            NaN
1146            NaN
Name: hull_kurt_area, Length: 1155, dtype: float64

In [8]:
main_df['intersect_roi'].value_counts()

intersect_roi
 1.0    600
 0.0    535
-1.0      1
Name: count, dtype: int64

In [9]:
save_cols = (relevant_cols +
        ['total_samples', 'beat_all_priors', 'best_prior', 'failure_category', 'failure_type', 'which_ones'])


In [10]:
main_df[save_cols].drop(['hull', 'hull_kurt'], axis=1).to_csv(save_path_without_hull)
pd.to_pickle(main_df[save_cols], save_path_with_hull)

In [11]:
main_df[(main_df["dataset"] == "pastis") & (main_df["transform"] == "fourier")]

,dataset,subset,transform,group,orientation,channel,dataset_type,obs_var,var_lower,var_upper,...,kstest_pval_gengamma,github_plot,best_beta,best_1/beta,beat_all_priors,best_prior,frequency,failure_category,failure_type,which_ones
626,pastis,full,fourier,2,NaN,green,remote sensing,10.047775,9.564716,10.551455,...,8.167974e-01,https://github.com/yashdave003/hierarchical-ba...,1.027027,0.973684,1,GenGamma,0.061242,actually_pass,pass,NaN
627,pastis,full,fourier,5,NaN,green,remote sensing,5.438562,5.180850,5.707798,...,8.517701e-01,https://github.com/yashdave003/hierarchical-ba...,1.197605,0.835000,1,GenGamma,0.078689,actually_pass,pass,NaN
628,pastis,full,fourier,8,NaN,green,remote sensing,3.018795,2.876773,3.165153,...,6.000704e-01,https://github.com/yashdave003/hierarchical-ba...,3.333333,0.300000,1,GenGamma,0.098453,actually_pass,pass,NaN
629,pastis,full,fourier,11,NaN,green,remote sensing,1.740533,1.654402,1.830307,...,6.841823e-01,https://github.com/yashdave003/hierarchical-ba...,31.000000,0.032258,1,GenGamma,0.123565,actually_pass,pass,NaN
630,pastis,full,fourier,14,NaN,green,remote sensing,0.964945,0.918967,1.013377,...,3.399267e-01,https://github.com/yashdave003/hierarchical-ba...,2.195122,0.455556,1,GenGamma,0.155566,actually_pass,pass,NaN
631,pastis,full,fourier,17,NaN,green,remote sensing,0.544932,0.518553,0.572532,...,9.951664e-01,https://github.com/yashdave003/hierarchical-ba...,5.937500,0.168421,1,GenGamma,0.196261,actually_pass,pass,NaN
632,pastis,full,fourier,20,NaN,green,remote sensing,0.277552,0.264042,0.291775,...,9.442287e-01,https://github.com/yashdave003/hierarchical-ba...,13.750000,0.072727,1,GenGamma,0.247131,actually_pass,pass,NaN
633,pastis,full,fourier,23,NaN,green,remote sensing,0.131865,0.125401,0.138744,...,2.176355e-01,https://github.com/yashdave003/hierarchical-ba...,24.000000,0.041667,1,GenGamma,0.311794,actually_pass,pass,NaN
634,pastis,full,fourier,26,NaN,green,remote sensing,0.054975,0.051767,0.058932,...,2.427087e-01,https://github.com/yashdave003/hierarchical-ba...,51.500000,0.019417,1,GenGamma,0.392401,actually_pass,pass,NaN
635,pastis,full,fourier,29,NaN,green,remote sensing,0.019339,0.017729,0.021846,...,2.313727e-02,https://github.com/yashdave003/hierarchical-ba...,47.500000,0.021053,1,GenGamma,0.493909,practically_pass,pass,NaN
